In [234]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from genetic_algorithm import GeneticAlgorithm

# Load the data
data = pd.read_csv(r"C:\Users\Karthick NG\Desktop\KARTHICK\SNU\4TH SEMESTER\1.ML\1.ML LAB\EX_9\Bank_Personal_Loan_Modelling.csv")


In [235]:

X = data.drop(['ID', 'ZIP Code', 'Personal Loan'], axis=1)
y = data['Personal Loan']
X = (X - X.mean()) / X.std()


In [236]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [237]:
model = Sequential()
model.add(Dense(64, input_shape=(10,), activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


In [238]:
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.01), metrics=['accuracy'])


In [239]:
def fitness_function(weights):
    model.set_weights(weights)
    loss, accuracy = model.evaluate(X_train, y_train, verbose=0)
    return accuracy


In [240]:
import numpy as np
from keras.models import clone_model

class GeneticAlgorithm:
    def __init__(self, model, param_space):
        self.model = model
        self.param_space = param_space
        self.population_size = self.param_space['population_size']
        self.elite_size = self.param_space['elite_size']
        self.num_generations = self.param_space['num_generations']
        self.mutation_rate = self.param_space['mutation_rate']
        self.crossover_rate = self.param_space['crossover_rate']
        self.num_weights = self.param_space['num_weights']
        self.fitness_func = self.param_space['fitness_func']
        self.generation_count = 0

        # Initialize the population
        self.population = self._initialize_population()

    def _initialize_population(self):
        population = []
        for i in range(self.population_size):
            weights = self.model.get_weights()
            for j in range(len(weights)):
                weights[j] = np.random.randn(*weights[j].shape)
            population.append(weights)
        return population

    def evolve(self):
        for i in range(self.num_generations):
            # Evaluate the fitness of the population
            fitness_scores = self._calculate_fitness()

            # Select the best individuals from the population
            elite_indices = self._select_elite(fitness_scores)
            elite_population = [self.population[i] for i in elite_indices]

            # Create a new population through crossover and mutation
            new_population = []
            while len(new_population) < self.population_size:
                parent1, parent2 = self._select_parents(fitness_scores)
                child1, child2 = self._crossover(parent1, parent2)
                child1 = self._mutate(child1)
                child2 = self._mutate(child2)
                new_population.append(child1)
                new_population.append(child2)

            # Replace the old population with the new population
            self.population = elite_population + new_population[:self.population_size - self.elite_size]

            # Increment the generation count
            self.generation_count += 1

        # Set the model weights to the best individual from the final generation
        best_individual = self._get_best_individual()
        self.model.set_weights(best_individual)

        # Return the best individual
        return best_individual

    def _calculate_fitness(self):
        fitness_scores = []
        for weights in self.population:
            self.model.set_weights(weights)
            fitness_scores.append(self.fitness_func(self.model))
        return np.array(fitness_scores)

    # Other methods as before


In [241]:
params = {
    'num_generations': 50,
    'population_size': 50,
    'elite_size': 5,
    'mutation_rate': 0.05,
    'crossover_rate': 0.8,
    'fitness_func': fitness_function,
    'num_weights': model.count_params()
}

# Create a GeneticAlgorithm object and run the genetic algorithm
ga = GeneticAlgorithm(model=model, param_space=params)


In [242]:
# weights = ga()


In [243]:
# model.set_weights(weights)
# _, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
# print('Test Accuracy:', test_accuracy)


In [244]:
# y_pred = model(X_test)


In [245]:
# y_pred = (y_pred>=0.5, 1, 0).flatten()

# print(classification_report(y_pred,test_y))

In [246]:

# X = data.drop(['ID', 'ZIP Code', 'Personal Loan'], axis=1)
# y = data['Personal Loan']

# X = pd.get_dummies(X, columns=['Family', 'Education', 'Securities Account', 'CD Account', 'Online', 'CreditCard'])

# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)


In [247]:
# # model = Sequential()
# model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [248]:
# # Import the GeneticAlgorithm class from the genetic_algorithm.py file
# from genetic_algorithm import GeneticAlgorithm

# # Define the fitness function
# def fitness_function(individual, x_train, y_train, x_test, y_test):
#     # Set the hyperparameters
# #     num_layers = individual['num_layers']
#     num_neurons = individual['num_neurons']
#     activation = individual['activation']
#     optimizer = individual['optimizer']

#     # Define the neural network architecture
#     model = Sequential()
#     model.add(Dense(num_neurons, input_dim=x_train.shape[1], activation=activation))
#     for i in range(num_layers-1):
#         model.add(Dense(num_neurons, activation=activation))
#     model.add(Dense(1, activation='sigmoid'))

#     # Compile the model
#     model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#     # Train the model on the training set
#     model.fit(x_train, y_train, epochs=10, batch_size=64, verbose=0)

#     # Evaluate the model on the test set and return the accuracy as fitness
#     _, accuracy = model.evaluate(x_test, y_test, verbose=0)
#     return accuracy

# # Define the search space for the hyperparameters
# search_space = {
#     'num_layers': [1, 2, 3],
#     'num_neurons': [32, 64, 128],
#     'activation': ['relu', 'sigmoid'],
#     'optimizer': ['adam', 'sgd']
# }

# # Create an instance of the GeneticAlgorithm class
# ga = GeneticAlgorithm()

# # Run the genetic algorithm
# best_individual = ga.run(fitness_function, x_train, y_train, x_test, y_test, search_space, 
#                          population_size=20, crossover_prob=0.8, mutation_prob=0.2, 
#                          tournament_size=3, elite_size=2, max_generations=10, 
#                          max_fitness_evals=1000)


In [249]:
# import random

# class GeneticAlgorithm:
#     def __init__(self, model, param_space):
#         self.model = model
#         self.param_space = param_space

#     def generate_population(self, population_size):
#         population = []
#         for i in range(population_size):
#             individual = {}
#             for param, values in self.param_space.items():
#                 individual[param] = random.choice(values)
#             population.append(individual)
#         return population

#     def evaluate_population(self, population, x_train, y_train, x_test, y_test):
#         fitness_scores = []
#         for individual in population:
#             fitness = self.fitness_function(individual, x_train, y_train, x_test, y_test)
#             fitness_scores.append((individual, fitness))
#         return fitness_scores

#     def select_parents(self, population, tournament_size):
#         parents = []
#         for i in range(2):
#             tournament = random.sample(population, tournament_size)
#             best_individual = max(tournament, key=lambda x: x[1])
#             parents.append(best_individual[0])
#         return parents

#     def crossover(self, parent1, parent2, crossover_prob):
#         if random.random() > crossover_prob:
#             return parent1, parent2
#         child1 = {}
#         child2 = {}
#         for param in self.param_space.keys():
#             if random.random() < 0.5:
#                 child1[param] = parent1[param]
#                 child2[param] = parent2[param]
#             else:
#                 child1[param] = parent2[param]
#                 child2[param] = parent1[param]
#         return child1, child2

#     def mutate(self, individual, mutation_prob):
#         for param, values in self.param_space.items():
#             if random.random() < mutation_prob:
#                 individual[param] = random.choice(values)
#         return individual

#     def run(self, fitness_function, x_train, y_train, x_test, y_test, 
#             population_size, crossover_prob, mutation_prob, tournament_size, 
#             elite_size, max_generations, max_fitness_evals):
#         population = self.generate_population(population_size)
#         num_evals = 0
#         for generation in range(max_generations):
#             fitness_scores = self.evaluate_population(population, x_train, y_train, x_test, y_test)
#             num_evals += population_size
#             best_individual = max(fitness_scores, key=lambda x: x[1])[0]
#             if num_evals >= max_fitness_evals:
#                 break
#             if best_individual['fitness'] >= 1.0:
#                 break
#             elite = [x[0] for x in sorted(fitness_scores, key=lambda x: x[1], reverse=True)[:elite_size]]
#             new_population = elite
#             while len(new_population) < population_size:
#                 parent1, parent2 = self.select_parents(population, tournament_size)
#                 child1, child2 = self.crossover(parent1, parent2, crossover_prob)
#                 child1 = self.mutate(child1, mutation_prob)
#                 child2 = self.mutate(child2, mutation_prob)
#                 new_population.append(child1)
#                 if len(new_population) < population_size:
#                     new_population.append(child2)
#             population = new_population
#         best_individual = max(self.evaluate_population([best_individual], x_train, y_train, x_test, y_test), 
#                               key=lambda x: x[1])[0]
#         return best_individual


In [250]:
# # Import the GeneticAlgorithm class from the genetic_algorithm.py file
# from genetic_algorithm import GeneticAlgorithm

# # Define the genetic algorithm parameters
# population_size = 20
# crossover_prob = 0.8
# mutation_prob = 0.2
# tournament_size = 3
# elite_size = 2
# max_generations = 10
# max_fitness_evals = 1000

# # Create an instance of the GeneticAlgorithm class
# ga = GeneticAlgorithm()

# # Run the genetic algorithm
# best_individual = ga.run(fitness_function, x_train, y_train, x_test, y_test, search_space, 
#                          population_size=population_size, crossover_prob=crossover_prob, 
#                          mutation_prob=mutation_prob, tournament_size=tournament_size, 
#                          elite_size=elite_size, max_generations=max_generations, 
#                          max_fitness_evals=max_fitness_evals)


In [251]:
# # Update the neural network model with the best individual
# num_layers = best_individual['num_layers']
# num_neurons = best_individual['num_neurons']
# activation = best_individual['activation']
# optimizer = best_individual['optimizer']

# model = Sequential()
# model.add(Dense(num_neurons, input_dim=x_train.shape[1], activation=activation))
# for i in range(num_layers-1):
#     model.add(Dense(num_neurons, activation=activation))
# model.add(Dense(1, activation='sigmoid'))

# model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# model.fit(x_train, y_train, epochs=100, batch_size=64, verbose=0)

# # Evaluate the model on the test set
# _, accuracy = model.evaluate(x_test, y_test, verbose=0)
# print("Accuracy on test set: {:.2f}%".format(accuracy * 100))


In [252]:

# # model_tuple = (model,)
# # Create a GeneticAlgorithm objaect and run the genetic algorithm
# ga = GeneticAlgorithm(model=model, param_space=params)


In [253]:

# # model_tuple = (model,)
# # Create a GeneticAlgorithm objaect and run the genetic algorithm
# ga = GeneticAlgorithm(model=model, param_space=params)


In [254]:

# model.set_weights(best_weights)
# _, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
# print('Test Accuracy:', test_accuracy)